In [ ]:
%pip install evaluate

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import evaluate
import torch
from tqdm import tqdm
import math

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/cl2-finalproj/sentiment
%ls

/content/drive/MyDrive/cl2-finalproj/sentiment
cs_sentiment/  english-sentiment/  sentiment-data-prep.ipynb  spanish-sentiment/  tmp_trainer/


# Preprocessing

In [ ]:
# CSV with columns: tweets, sentiment
cs_train = pd.read_csv("cs_sentiment/train.tsv", sep='\t').sample(1800)
# eng_train = pd.read_csv("english-sentiment/train.csv").sample(1800)
# spa_train = pd.read_csv("spanish-sentiment/train.csv").sample(1800)
# bi_train = pd.concat([eng_train[:900],spa_train[:900]]).sample(1800).reset_index()

cs_test = pd.read_csv("cs_sentiment/test.tsv", sep='\t').sample(800)
# eng_test = pd.read_csv("english-sentiment/test.csv").sample(800)
# spa_test = pd.read_csv("spanish-sentiment/test.csv").sample(800)
# bi_test = pd.concat([eng_test[:900],spa_test[:900]]).sample(800).reset_index()

In [ ]:
# trains = [cs_train, eng_train, spa_train, bi_train]
# tests = [cs_test, eng_test, spa_test, bi_test]

trains = [cs_train]
tests = [cs_test]

In [ ]:
def senti_to_num(senti):
    if senti == "positive":
        return 2
    elif senti == "neutral":
        return 1
    elif senti == "negative":
        return 0

# for train in trains:
#     train["label"] = train["label"].apply(lambda x: senti_to_num(x))

# for test in tests:
#     test["label"] = test["label"].apply(lambda x: senti_to_num(x))

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased")
# model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-multilingual-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
# curr_max = 0
# for train in trains:
#     max_len = max(train['text'].apply(lambda x: len(x)))
#     if curr_max < max_len:
#         curr_max = max_len
# max_len = curr_max
max_len = 152

# Functions

In [ ]:
metric = evaluate.load("accuracy")

def tokenize(tokenizer, example, max_len):
    toks = tokenizer(example['text'], padding="max_length", truncation=True, max_length=max_len+15)
    toks.update({"labels": example['label']})
    return toks

def compute_metrics(metric, eval_pred):
    # TODO: Double check this
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def add_tok(df, tokenizer, max_len):
    df["toked"] = df.apply(lambda x: tokenize(tokenizer, x, max_len), axis=1)
    ds = Dataset.from_pandas(pd.DataFrame(list(df['toked'])))
    return ds

def prep_df(train_df, test_df, tokenizer, max_len):
    train_ds = add_tok(train_df, tokenizer, max_len)
    test_ds = add_tok(test_df, tokenizer, max_len)
    return train_ds, test_ds

In [ ]:
pd.read_csv("english-sentiment/train.csv").sample(1800).head()

,text,label,source,toked
8936,@_mgmiranda hahahaha a mí me pasa al revés !! ...,0,LINCE,"[input_ids, token_type_ids, attention_mask, la..."
8612,RT @EmeraudeFans : beautiful @EmeraudeToubia c...,2,LINCE,"[input_ids, token_type_ids, attention_mask, la..."
6363,Estas rolas viejitas got me feelin some kinda ...,2,LINCE,"[input_ids, token_type_ids, attention_mask, la..."
5096,50$ por un VIP lap dance,1,LINCE,"[input_ids, token_type_ids, attention_mask, la..."
6338,No mames pinche autocorrect culoo ✊,0,LINCE,"[input_ids, token_type_ids, attention_mask, la..."


In [ ]:
# def main_train(train_df, test_df, tokenizer, model, name, max_len, optimizer, metric, epochs=10):


# CSV with columns: tweets, sentiment
# cs_train = pd.read_csv("cs_sentiment/train.tsv", sep='\t').sample(1800)
# eng_train = pd.read_csv("english-sentiment/train.csv").sample(1800)
# spa_train = pd.read_csv("spanish-sentiment/train.csv").sample(1800)
# train_df = pd.concat([eng_train[:900],spa_train[:900]]).sample(1800).reset_index()


# Change train_df and name only

# CHANGE BELOW
train_df = pd.read_csv("english-sentiment/train.csv").sample(1800)
name = "english2"
# CHANGE ABOVE


train_df["label"] = train_df["label"].apply(lambda x: senti_to_num(x))
# test_df = pd.read_csv("english-sentiment/test.csv").sample(800)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
optimizer = torch.optim.AdamW(
    model.parameters(),
    eps = 1e-08,
    lr = 1e-4,
)

epochs = 10
    # ------------------------------------------------------------------------------------------------------------------
train_ds = add_tok(train_df, tokenizer, max_len)
batch_size = 32
train_dataloader = DataLoader(
    train_ds,
    batch_size=batch_size,
    collate_fn=lambda samples: {
        "input_ids": torch.tensor([sample["input_ids"] for sample in samples]),
        "attention_masks": torch.tensor([sample["attention_mask"] for sample in samples]),
        "labels": torch.tensor([sample["labels"] for sample in samples])
    }
)
# test_dataloader = DataLoader(
#     test_ds,
#     batch_size=batch_size,
#     collate_fn=lambda samples: {
#         "input_ids": torch.tensor([sample["input_ids"] for sample in samples]),
#         "attention_masks": torch.tensor([sample["attention_mask"] for sample in samples]),
#         "labels": torch.tensor([sample["labels"] for sample in samples])
#     }
# )
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
for ep in range(0, epochs):
    print("Current epoch: " + str(ep))

    # ========== Training ==========

    # Set model to training mode
    model.train()

    for step, batch in enumerate(tqdm(train_dataloader)):
        b_input_ids, b_input_mask, b_labels = batch['input_ids'].to(device), batch['attention_masks'].to(device), batch['labels'].to(device)
        optimizer.zero_grad()

        # Forward pass
        train_output = model(
            b_input_ids,
            attention_mask = b_input_mask,
            labels = b_labels
        )

        # Backward pass
        train_output.loss.backward()
        optimizer.step()

torch.save(model.state_dict(), "/content/drive/MyDrive/cl2-finalproj/" + name + '.pth')
model.save_pretrained("/content/drive/MyDrive/cl2-finalproj/" + name)

# model.eval()
# all_preds = np.array([])
# all_labels = np.array([])

# for batch in test_dataloader:
#     batch = tuple(t.to(device) for t in batch)
#     b_input_ids, b_input_mask, b_labels = batch
#     with torch.no_grad():
#         # Forward pass
#         eval_output = model(b_input_ids,
#                             token_type_ids = None,
#                             attention_mask = b_input_mask)

#     logits = eval_output.logits.detach().cpu().numpy()
#     predictions = np.argmax(logits, axis=-1)
#     label_ids = b_labels.to('cpu').numpy()

#     all_preds = np.concatenate([all_preds, predictions])
#     all_labels = np.concatenate([all_labels, label_ids])

    # return metric.compute(predictions=all_preds, references=all_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Current epoch: 0


100%|██████████| 57/57 [00:48<00:00,  1.17it/s]


Current epoch: 1


100%|██████████| 57/57 [00:47<00:00,  1.19it/s]


...

100%|██████████| 57/57 [00:47<00:00,  1.19it/s]


Current epoch: 8


100%|██████████| 57/57 [00:47<00:00,  1.19it/s]


Current epoch: 9


100%|██████████| 57/57 [00:47<00:00,  1.19it/s]


# Main Loop

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
optimizer = torch.optim.AdamW(
    model.parameters(),
    eps = 1e-08,
    lr = 1e-4,
)

# change train/test datasets and names: ["codeswitched", "english", "spanish", "bilingual"]
results = main_train(cs_train, cs_test, tokenizer, model, "codeswitched", max_len, optimizer, metric)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Current epoch: 0


100%|██████████| 57/57 [00:45<00:00,  1.24it/s]


Current epoch: 1


100%|██████████| 57/57 [00:45<00:00,  1.25it/s]


...

Current epoch: 8


100%|██████████| 57/57 [00:45<00:00,  1.25it/s]


Current epoch: 9


100%|██████████| 57/57 [00:45<00:00,  1.25it/s]


RuntimeError: Parent directory drive/MyDrive/cl2-finalproj does not exist.

In [ ]:
# names = ["cs", "eng", "spa", "bilin"]
# all_res = []
# for i in range(0, len(names)):
#     model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
#     optimizer = torch.optim.AdamW(
#         model.parameters(),
#         eps = 1e-08,
#         lr = 1e-4,
#     )
#     results = main_train(trains[i], tests[i], tokenizer, model, names[i], max_len, optimizer, metric)
#     all_res.append(results)

In [ ]:
%ls

cs_sentiment/  english-sentiment/  sentiment-data-prep.ipynb  spanish-sentiment/  tmp_trainer/


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
curr_model.eval()

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [ ]:
cs_test = pd.read_csv("cs_sentiment/test.tsv", sep='\t').sample(800)
eng_test = pd.read_csv("english-sentiment/test.csv").sample(800)
spa_test = pd.read_csv("spanish-sentiment/test.csv").sample(800)
bi_test = pd.concat([eng_test[:900],spa_test[:900]]).sample(800).reset_index()

combined = pd.concat([cs_test, eng_test, spa_test, bi_test])

test_datasets = [cs_test, eng_test, spa_test, bi_test]

for test in test_datasets:
    test["label"] = test["label"].apply(lambda x: senti_to_num(x))

model_names = ["code_switched2", "english2", "spanish", "bilingual"]

model_names = ["english2"]

def compute_metrics(predictions, references):
    # metric1 = evaluate.load("precision")
    # metric2 = evaluate.load("recall")
    # metric3 = evaluate.load("f1")
    # metric4 = evaluate.load("accuracy")

    # precision = metric1.compute(predictions=predictions, references=references, average="micro")["precision"]
    # recall = metric2.compute(predictions=predictions, references=references, average="micro")["recall"]
    # f1 = metric3.compute(predictions=predictions, references=references, average="micro")["f1"]
    # accuracy = metric4.compute(predictions=predictions, references=references)["accuracy"]
    loss = 0
    losssquared = 0

    for i in range(0, len(predictions)):
        loss += abs(references[i] - predictions[i])
        losssquared += (references[i] - predictions[i])**2

    loss /= len(predictions)
    losssquared /= len(predictions)

    return {"loss": loss, "losssquared": losssquared}

def testing_on_all(modelname, test_datasets, tokenizer, max_len, batch_size=32):
    state_dict = torch.load("/content/drive/MyDrive/cl2-finalproj/" + modelname + '.pth')
    curr_model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
    curr_model.load_state_dict(state_dict)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    curr_model.to(device)
    all_results = []
    for t_ds in test_datasets:
        t_ds = add_tok(t_ds, tokenizer, max_len)
        test_dataloader = DataLoader(
            t_ds,
            batch_size=batch_size,
            collate_fn=lambda samples: {
                "input_ids": torch.tensor([sample["input_ids"] for sample in samples]),
                "attention_masks": torch.tensor([sample["attention_mask"] for sample in samples]),
                "labels": torch.tensor([sample["labels"] for sample in samples])
            }
        )

        curr_model.eval()
        all_preds = np.array([])
        all_labels = np.array([])

        for batch in test_dataloader:
            batch = tuple(batch[t].to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():
                # Forward pass
                eval_output = curr_model(b_input_ids,
                                    attention_mask = b_input_mask)

            predictions = eval_output.logits.detach().cpu().numpy()
            # predictions = np.argmax(logits, axis=-1)
            label_ids = b_labels.to('cpu').numpy()
            preds = []
            for pred in predictions:
                total = math.e**pred[0] + math.e**pred[1] + math.e**pred[2]
                negative = math.e**pred[0] * -1
                neutral = 0
                positive = math.e**pred[2] * 1
                all = negative/total + positive/total + 1
                preds.append(all)
            preds = np.array(preds)

            print(predictions)
            break

            all_preds = np.concatenate([all_preds, preds])
            all_labels = np.concatenate([all_labels, label_ids])
        all_results.append(compute_metrics(all_preds, all_labels))
    return all_results

mod_results = []
for mod in model_names:
    mod_results.append(mod)
    results = testing_on_all(mod, test_datasets, tokenizer, max_len)
    mod_results.append(results)

print(mod_results)

<ipython-input-10-3de167ae9e7e>:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/content/drive/MyDrive/cl2-finalproj/" + modelname + '.pth')
Some 

[[-0.18390806 -0.17598346 -0.15810958]
 [-0.18390799 -0.17598315 -0.15810989]
 [-0.1839079  -0.17598398 -0.15810929]
 [-0.18390863 -0.17597762 -0.15811391]
 [-0.18390757 -0.17598638 -0.15810771]
 [-0.18390796 -0.17598388 -0.15810944]
 [-0.183908   -0.17598327 -0.15810965]
 [-0.18390799 -0.17598417 -0.15810905]
 [-0.183908   -0.1759841  -0.1581091 ]
 [-0.18390822 -0.17598154 -0.1581109 ]
 [-0.18390763 -0.17598602 -0.15810779]
 [-0.18390863 -0.17597829 -0.15811339]
 [-0.18390751 -0.17598814 -0.15810637]
 [-0.18390803 -0.17598322 -0.15810995]
 [-0.18390808 -0.17598207 -0.15811057]
 [-0.18390775 -0.17598516 -0.15810834]
 [-0.18390825 -0.17598206 -0.15811047]
 [-0.18390813 -0.17598338 -0.15810965]
 [-0.18390805 -0.17598356 -0.15810964]
 [-0.1839079  -0.175984   -0.15810923]
 [-0.18390794 -0.1759839  -0.15810938]
 [-0.18390796 -0.17598371 -0.15810966]
 [-0.1839087  -0.17597763 -0.15811384]
 [-0.18390818 -0.1759811  -0.15811121]
 [-0.18390779 -0.17598464 -0.15810883]
 [-0.18390825 -0.17598148

ZeroDivisionError: division by zero

In [ ]:
bi_test

,index,text,label,source,toked
0,806,Russia: Zenit stadium will be finished ‘by the...,2,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
1,298,A Cartoon from The New Yorker via @user #Trum...,1,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
2,642,Uno de los peores dias,0,intertass_2017,"[input_ids, token_type_ids, attention_mask, la..."
3,768,Apparently those neo-Nazi’s didn’t actually ME...,0,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
4,756,Dylann Roof competent to stand trial. Time to ...,0,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
...,...,...,...,...,...
795,809,"...y todo lo que hagan, háganlo con amor. 1 Co...",2,intertass_2017,"[input_ids, token_type_ids, attention_mask, la..."
796,648,@user me siento very important jeje no en verd...,0,intertass_2017,"[input_ids, token_type_ids, attention_mask, la..."
797,366,@user and rightists are incapable of learning....,0,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
798,578,"@user Jajaja es muy triste esto, pero ya hoy c...",2,intertass_2017,"[input_ids, token_type_ids, attention_mask, la..."


In [ ]:
pd.DataFrame(bi_test["source"]).value_counts()

,count
source,
sem_eval_2017,406
intertass_2017,394


In [ ]:
t_ds = add_tok(cs_test, tokenizer, max_len)
test_dataloader = DataLoader(
    t_ds,
    batch_size=32,
    collate_fn=lambda samples: {
        "input_ids": torch.tensor([sample["input_ids"] for sample in samples]),
        "attention_masks": torch.tensor([sample["attention_mask"] for sample in samples]),
        "labels": torch.tensor([sample["labels"] for sample in samples])
    }
)

In [ ]:
for batch in test_dataloader:
    x = batch
    batch = tuple(t.to(device) for t in batch)

AttributeError: 'str' object has no attribute 'to'

In [ ]:
for t in batch:
    print(t)
    break

input_ids


In [ ]:
batch

{'input_ids': tensor([[  101, 97200, 19047,  ...,     0,     0,     0],
         [  101,   144, 10107,  ...,     0,     0,     0],
         [  101, 19497, 10104,  ...,     0,     0,     0],
         ...,
         [  101, 96672, 10133,  ...,     0,     0,     0],
         [  101, 56898,   137,  ...,     0,     0,     0],
         [  101, 56898,   137,  ...,     0,     0,     0]]),
 'attention_masks': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2,
         1, 2, 2, 2, 0, 2, 2, 1])}

In [ ]:
tok

In [ ]:
cs_test.iloc[0]["toked"]

{'input_ids': [101, 97200, 19047, 10229, 10121, 47430, 67292, 10731, 22536, 10196, 150, 110903, 10731, 100, 100, 65689, 10354, 109, 10186, 119, 10919, 108, 31119, 67929, 63143, 53504, 10376, 108, 28552, 10354, 108, 63938, 14985, 10556, 131, 120, 120, 188, 119, 11170, 120, 122, 12022, 100673, 37462, 11166, 10457, 11211, 11779, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
train_df = pd.read_csv("english-sentiment/train.csv").sample(1800)

train_df["label"] = train_df["label"].apply(lambda x: senti_to_num(x))
train_ds = add_tok(train_df, tokenizer, max_len)

train_dataloader = DataLoader(
    train_ds,
    batch_size=32,
    collate_fn=lambda samples: {
        "input_ids": torch.tensor([sample["input_ids"] for sample in samples]),
        "attention_masks": torch.tensor([sample["attention_mask"] for sample in samples]),
        "labels": torch.tensor([sample["labels"] for sample in samples])
    }
)

In [ ]:
train_df

,text,label,source,toked
191,@user @user @user @user I\u2019d watch Cougar ...,2,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
322,Selling a ticket for Twilight Sad at the Barro...,1,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
1423,Im in the mood to watch Next Friday but Netfli...,1,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
710,@user @user Lil Kim will be remembered as the ...,2,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
1709,@user is true that you will give two concerts ...,2,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
...,...,...,...,...
97,@user Arsenal have won three PL games at the E...,1,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
1447,LG Spectrum Ice Cream Sandwich will start roll...,1,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
172,"You may think Venus and Serena are impressive,...",1,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."
969,I just saw a 3rd grader talking on her I phone...,0,sem_eval_2017,"[input_ids, token_type_ids, attention_mask, la..."


In [ ]:
for step, batch in enumerate(tqdm(train_dataloader)):
    print(batch)
    y = batch
    this = tuple(t.to(device) for t in batch)
    break

  0%|          | 0/57 [00:00<?, ?it/s]

{'input_ids': tensor([[  101,   137, 29115,  ...,     0,     0,     0],
        [  101, 11045, 30295,  ...,     0,     0,     0],
        [  101, 10796, 10106,  ...,     0,     0,     0],
        ...,
        [  101,   107, 96906,  ...,     0,     0,     0],
        [  101,   107, 51768,  ...,     0,     0,     0],
        [  101,   137, 29115,  ...,     0,     0,     0]]), 'attention_masks': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([2, 1, 1, 2, 2, 2, 2, 2, 0, 1, 1, 1, 2, 1, 1, 2, 2, 1, 0, 0, 0, 0, 1, 0,
        2, 1, 0, 2, 2, 0, 2, 0])}


AttributeError: 'str' object has no attribute 'to'

tensor([[  101,   137, 29115,  ...,     0,     0,     0],
        [  101, 11045, 30295,  ...,     0,     0,     0],
        [  101, 10796, 10106,  ...,     0,     0,     0],
        ...,
        [  101,   107, 96906,  ...,     0,     0,     0],
        [  101,   107, 51768,  ...,     0,     0,     0],
        [  101,   137, 29115,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([2, 1, 1, 2, 2, 2, 2, 2, 0, 1, 1, 1, 2, 1, 1, 2, 2, 1, 0, 0, 0, 0, 1, 0,
        2, 1, 0, 2, 2, 0, 2, 0])
